<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Breakout_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-07-22 09:18:03


In [2]:
!pip install yfinance

In [3]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yfin
#import talib
yfin.pdr_override()
#import pandas_ta as ta
import pandas as pd

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

In [5]:
def get_breakout(stock_symbol):

  stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close'])]
  stock_df['Stock'] = stock_symbol
  stock_df['Open'] = round(stock_df['Open'],0)
  stock_df['Close'] = round(stock_df['Close'],0)
  stock_df = stock_df.reset_index()
  prev_max = max(stock_df[(stock_df['Date'] < '2023-01-01')]['Close'])
  stock_df['PrevMax'] = prev_max
  tmp_df = stock_df[(stock_df['Close'] > prev_max)]
  tmp_df['DateGroup'] = tmp_df['Date'].diff().dt.days.ne(1).cumsum()
  tmp_df['Date'] = tmp_df['Date'].dt.strftime('%Y-%m-%d')
  tmp_df = tmp_df[tmp_df['Close'] > tmp_df['Open']]
  g = tmp_df.groupby('DateGroup')['Date'].agg(['min', 'max', 'count']).reset_index()

  df_res = tmp_df.merge(g, how='left')
  return df_res[df_res['count'] > 1].head(1)

In [7]:
df_breakout = pd.DataFrame()

for stock_id in stock_ids:
  tmp_df = get_breakout(stock_id+'.NS')
  df_breakout = df_breakout.append(tmp_df)

df_breakout.sort_values(by = 'Date', ascending=False)

,Date,Open,Close,Stock,PrevMax,DateGroup,min,max,count
0,2023-07-18,2817.0,2820.0,RELIANCE.NS,2811.0,1,2023-07-18,2023-07-19,2
0,2023-07-18,15709.0,15896.0,PGHH.NS,15778.0,1,2023-07-18,2023-07-20,3
2,2023-07-10,1036.0,1045.0,SUNPHARMA.NS,1044.0,3,2023-07-10,2023-07-12,3
4,2023-07-10,786.0,807.0,JSWSTEEL.NS,766.0,6,2023-07-10,2023-07-12,2
1,2023-07-10,1770.0,1779.0,COLPAL.NS,1719.0,2,2023-07-10,2023-07-14,4
2,2023-07-06,864.0,870.0,BHARTIARTL.NS,851.0,4,2023-07-06,2023-07-07,2
2,2023-07-05,22867.0,23020.0,ABBOTINDIA.NS,22611.0,3,2023-07-05,2023-07-06,2
1,2023-07-05,942.0,958.0,ICICIBANK.NS,953.0,2,2023-07-05,2023-07-06,2
2,2023-07-04,395.0,403.0,UJJIVAN.NS,395.0,3,2023-07-04,2023-07-05,2
0,2023-07-04,321.0,322.0,CANBK.NS,321.0,2,2023-07-04,2023-07-07,3
